In [ ]:
# Set your TAVILY_API_KEY
!pip install langchain-tavily
import os
from langchain_tavily import TavilySearch

os.environ["TAVILY_API_KEY"] = "TAVILY_API_KEY"  # replace with actual key
tavily_tool = TavilySearch(max_results=2)

# Google Calendar setup
from googleapiclient.discovery import build
from google.oauth2 import service_account
from langchain.tools import tool

SERVICE_ACCOUNT_FILE = '3.json'  # your service account file
SCOPES = ['https://www.googleapis.com/auth/calendar']

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_service = build('calendar', 'v3', credentials=creds)

@tool
def calendar_tool(query: str) -> str:
    """Creates a calendar event (static example for now)."""
    event = {
        'summary': 'Test Event',
        'start': {'dateTime': '2025-05-15T10:00:00+05:00'},
        'end': {'dateTime': '2025-05-15T11:00:00+05:00'},
    }
    try:
        created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
        return f"Calendar event created: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating calendar event: {e}"


In [ ]:
# Define chatbot state and LangGraph setup
from typing import TypedDict, Literal, Union
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI

class ChatState(TypedDict):
    query: str
    tool_choice: Literal["search", "calendar", "chat"]
    result: Union[str, None]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)


In [ ]:
# Tool routing and logic
def route_tool(state: ChatState) -> ChatState:
    prompt = f"""You are a smart router. Classify the user query into one of the following:
    1. 'search' for web queries (e.g., 'Who won the match yesterday?')
    2. 'calendar' for events (e.g., 'Create a meeting tomorrow at 5 PM')
    3. 'chat' for general AI conversation

    Query: "{state['query']}"
    Answer with one word only: search, calendar, or chat.
    """
    decision = llm.invoke(prompt).content.strip().lower()
    return {"query": state["query"], "tool_choice": decision, "result": None}

def use_search(state: ChatState) -> ChatState:
    result = tavily_tool.invoke(state["query"])  # Tavily here
    return {"query": state["query"], "tool_choice": "search", "result": result}

def use_calendar(state: ChatState) -> ChatState:
    result = calendar_tool.invoke(state["query"])
    return {"query": state["query"], "tool_choice": "calendar", "result": result}

def chat_response(state: ChatState) -> ChatState:
    result = llm.invoke(state["query"]).content
    return {"query": state["query"], "tool_choice": "chat", "result": result}


In [ ]:
# LangGraph structure
graph_builder = StateGraph(ChatState)
graph_builder.set_entry_point("router")

graph_builder.add_node("router", route_tool)
graph_builder.add_node("search", use_search)
graph_builder.add_node("calendar", use_calendar)
graph_builder.add_node("chat", chat_response)
graph_builder.add_node("end", lambda x: x)

graph_builder.add_conditional_edges("router", lambda s: s["tool_choice"], {
    "search": "search",
    "calendar": "calendar",
    "chat": "chat"
})
graph_builder.add_edge("search", "end")
graph_builder.add_edge("calendar", "end")
graph_builder.add_edge("chat", "end")

graph = graph_builder.compile()


In [ ]:
# Interactive loop
def chatbot_interface(user_input):
    state = {"query": user_input, "tool_choice": "chat", "result": None}
    result = graph.invoke(state)
    return result["result"]

print("Chatbot started! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chatbot_interface(user_input)
    print(f"Bot: {response}")
